خوندن csv

In [2]:
import pandas as pd
#---------------read CSVs---------------
main_data = pd.read_csv("D:/venv1/drive-download-20250527T074916Z-1-001/Divar.csv")
cities_type = pd.read_csv("D:/venv1/drive-download-20250527T074916Z-1-001/iran_city_classification.csv")

C:\Users\ghase\AppData\Local\Temp\ipykernel_17576\1227722073.py:3: DtypeWarning: Columns (11,27,29,53) have mixed types. Specify dtype option on import or set low_memory=False.
  main_data = pd.read_csv("D:/venv1/drive-download-20250527T074916Z-1-001/Divar.csv")


ایجاد ستونی جدید و مشخص کردن اینکه آیا خانه در کلان شهر است یا خیر

In [3]:
#---------------Big cities---------------
big_cities_name = []
for j in cities_type[cities_type['دسته\u200cبندی']== 'کلان‌شهر']['نام شهر'].values:
    big_cities_name.append(j)
print(big_cities_name)

['karaj', 'tehran', 'mashhad', 'ahvaz', 'kermanshah', 'shiraz', 'tabriz', 'isfahan', 'qom']


In [4]:
#---------------add a new column about being big city or not---------------
main_data['big_city'] = main_data['city_slug'].isin(big_cities_name).astype(int)

عوض کردن تایپ ستون تاریخ ایجاد آگهی

In [5]:
#---------------change type of the created_at_month column---------------
main_data['created_at_month'] = pd.to_datetime(main_data['created_at_month'])

حل مشکل نشون دادن کارکتر‌های کمتر برای ستون توضیحات

In [6]:
#---------------Fixed the issue of printing fewer words in the description column---------------
pd.options.display.max_colwidth = 300

ستون شهر ها دو تا خالی داره 
میخوایم سطرهاشون رو پیدا کنیم و  ببینیم میشه از ستون توصیف شهر رو پیدا کرد یا نه

In [7]:
#---------------finding the rows that have null in city_slug column---------------
main_data[main_data['city_slug'].isnull()]

,Unnamed: 0,cat2_slug,cat3_slug,city_slug,neighborhood_slug,created_at_month,user_type,description,title,rent_mode,...,regular_person_capacity,extra_person_capacity,cost_per_extra_person,rent_price_on_regular_days,rent_price_on_special_days,rent_price_at_weekends,location_latitude,location_longitude,location_radius,big_city
292674,292674,commercial-rent,office-rent,NaN,NaN,2024-06-01,NaN,مجموعه ما تصمیم دارد جهت تکمیل کادر در کلینیک تخصصی پوست و آرامش واقع در گلسار ، از افرادعلاقمند در مشاغل زیربا داشتن ۲ سال سابقه کاردعوت به همکاری نمایید :ماسور حرفه ای - مژه کار - میکروبلیدینگ - تراپیست مو - ناخن کار و پدیکوریست\n(مخصوص بانوان),اجاره ی اتاق در مرکز زیبایی,مقطوع,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
676502,676502,residential-sell,house-villa-sell,NaN,NaN,2024-06-01,NaN,باغ ویلا بسیار شیک و خوش نقشه با موقعیت عالی سالن بزرگ ، چهار خواب ، مستر ، بالاکن های بزرگ ، استخر سر پوشیده ، سونا ، جکوزی ، موتور خانه ، زمین چمن مصنوعی \nدارای ساختمان سرایداری مستقل\nقابل معاوضه با زمین یا آپارتمان در شیراز,فروش باغ ویلا در قلات,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


میبینیم که اسم شهر ها در ستون توصیف اومده حالا باید جای خالی رو پر کنیم

In [8]:
#---------------filling the nulls---------------
main_data.loc[676502, 'city_slug'] = 'shiraz'
main_data.loc[676502, 'neighborhood_slug'] = 'golsar'

main_data.loc[292674, 'city_slug'] = 'shiraz'
main_data.loc[292674, 'neighborhood_slug'] = 'ghallat'


پر کردن مقادیر گمشده در ستون محله(در شهرهای کوچیک با نام شهر و در کلان شهرها با "نامشخص")

In [9]:
#---------------fill neighborhood_slug nulls for big_city == 0---------------
change_neighborhood = main_data['neighborhood_slug'].isnull() & (main_data['big_city'] == 0)
main_data.loc[change_neighborhood, 'neighborhood_slug'] = main_data.loc[change_neighborhood, 'city_slug']


In [10]:
main_data[main_data['big_city']==1]['neighborhood_slug'].value_counts()

neighborhood_slug
elahiyehblvd         7076
poonak               5393
ghasemabad           5216
saadat-abad          4686
bolvartoos           4684
                     ... 
kuy-azadi               2
qasr-e-firooze-do       2
baq-e-rezavan           2
sanjavamareh            2
shahrak-zafar           1
Name: count, Length: 1039, dtype: int64

In [11]:
#---------------fill neighborhood_slug nulls for big_city == 1---------------
change_neighborhood = main_data['neighborhood_slug'].isnull() & (main_data['big_city'] == 1)

main_data.loc[change_neighborhood, 'neighborhood_slug'] = 'unknown'

In [12]:
main_data[main_data['big_city']==1]['neighborhood_slug'].value_counts()

neighborhood_slug
unknown              44703
elahiyehblvd          7076
poonak                5393
ghasemabad            5216
saadat-abad           4686
                     ...  
baq-e-rezavan            2
kuy-azadi                2
sanjavamareh             2
qasr-e-firooze-do        2
shahrak-zafar            1
Name: count, Length: 1040, dtype: int64

به طور تقریبی 60 درصد زیربنا میشه ساختمون

In [13]:
#---------------fill land_size nulls base on building_size---------------
fill_land = (main_data['land_size'].isna()) & (main_data['building_size'].notna())

main_data.loc[fill_land, 'land_size'] = (main_data.loc[fill_land, 'building_size'] * 100) / 60

بقیه داده های نال توی ستون های اندازه زمین و ساختمان را نمیشه حساب کرد

In [14]:
#---------------fill other nulls for land_size and building_size---------------
fii_other_nulls_land = main_data['land_size'].isna()
main_data.loc[fii_other_nulls_land, 'land_size'] = -1

fii_other_nulls_building = main_data['building_size'].isna()
main_data.loc[fii_other_nulls_building, 'building_size'] = -1

این ستون تنها یک مقدار خالی داشته که با بررسی ستون توضیحات، مقدار پیدا شده پس سطر خالی را پر میکنیم

In [15]:
#---------------fill cat3_slug null---------------
main_data.loc[338801, 'cat3_slug'] = 'office-rent'

In [16]:
main_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 62 columns):
 #   Column                      Non-Null Count    Dtype         
---  ------                      --------------    -----         
 0   Unnamed: 0                  1000000 non-null  int64         
 1   cat2_slug                   1000000 non-null  object        
 2   cat3_slug                   1000000 non-null  object        
 3   city_slug                   1000000 non-null  object        
 4   neighborhood_slug           1000000 non-null  object        
 5   created_at_month            1000000 non-null  datetime64[ns]
 6   user_type                   288882 non-null   object        
 7   description                 1000000 non-null  object        
 8   title                       999946 non-null   object        
 9   rent_mode                   352994 non-null   object        
 10  rent_value                  351322 non-null   float64       
 11  rent_to_single           

آیتم‌های لاکچری در صورت دارا بودن

In [17]:
#---------------fill sauna_true---------------
sauna_true = main_data['description'].str.contains(r'\bسونا\b', regex=True, na=False) & (main_data['has_sauna'].isnull())
main_data.loc[sauna_true, 'has_sauna'] = True

#---------------fill jacuzzi_true---------------
jacuzzi_true = main_data['description'].str.contains(r'\bجکوزی\b', regex=True, na=False) & (main_data['has_jacuzzi'].isnull())
main_data.loc[jacuzzi_true, 'has_jacuzzi'] = True

#---------------fill pool_true---------------
pool_true = main_data['description'].str.contains(r'\bاستخر\b', regex=True, na=False) & (main_data['has_pool'].isnull())
main_data.loc[pool_true, 'has_pool'] = True

#---------------fill barbecue_true---------------
barbecue_true = main_data['description'].str.contains(r'\bباربیکیو\b', regex=True, na=False) & (main_data['has_barbecue'].isnull())
main_data.loc[barbecue_true, 'has_barbecue'] = True

#---------------fill security_guard_true---------------
security_guard_true = ((main_data['description'].str.contains(r'\bنگهبان\b', regex=True, na=False)) | (main_data['description'].str.contains(r'\bنگهبانی\b', regex=True, na=False)) | (main_data['description'].str.contains(r'\bلابی من\b', regex=True, na=False))) & (main_data['has_security_guard'].isnull())
main_data.loc[security_guard_true, 'has_security_guard'] = True


پر کردن مقادیر ستون‌ سند تجاری در صورت دارا بودن

In [18]:
#---------------has business_deed---------------
has_business_deed_true = main_data['description'].str.contains(r'\bسند تجاری\b', regex=True, na=False) & (main_data['has_business_deed'].isnull())
main_data.loc[has_business_deed_true, 'has_business_deed'] = True


پر کردن ستون بازسازی در صورت بازسازی شدن

In [19]:
#---------------true is_rebuilt---------------
rebuilt_true = main_data['description'].str.contains(r'\bبازسازی\b', regex=True, na=False) & (main_data['is_rebuilt'].isnull())
main_data.loc[rebuilt_true, 'is_rebuilt'] = True

پر کردن مقادیر ستون‌های بالکن و آسانسور و پارکینگ و انباری در صورت دارا بودن

In [20]:
#---------------true has_balcony----------------
security_guard_true = ((main_data['description'].str.contains(r'\bتراس\b', regex=True, na=False)) | (main_data['description'].str.contains(r'\bبالکن\b', regex=True, na=False))) & (main_data['has_balcony'].isnull())
main_data.loc[security_guard_true, 'has_balcony'] = True

#---------------true has_elevator----------------
has_business_deed_true = main_data['description'].str.contains(r'\bآسانسور\b', regex=True, na=False) & (main_data['has_elevator'].isnull())
main_data.loc[has_business_deed_true, 'has_elevator'] = True

#---------------true has_warehouse---------------
warehouse_true = ((main_data['description'].str.contains(r'\bانبار\b', regex=True, na=False)) | (main_data['description'].str.contains(r'\bانباری\b', regex=True, na=False))) & (main_data['has_warehouse'].isnull())
main_data.loc[warehouse_true, 'has_warehouse'] = True

#---------------true has_parking---------------
parking_true = main_data['description'].str.contains(r'\bپارکینگ\b', regex=True, na=False) & (main_data['has_parking'].isnull())
main_data.loc[parking_true, 'has_parking'] = True

پرکردن ستون‌های آب و برق و گاز در صورت دارا بودن

In [21]:
#---------------true has_water---------------
water_true = ((main_data['description'].str.contains(r'\bآب\b', regex=True, na=False)) | (main_data['description'].str.contains(r'\bاب\b', regex=True, na=False)) | (main_data['description'].str.contains(r'\bسونا\b', regex=True, na=False)) | (main_data['description'].str.contains(r'\bجکوزی\b', regex=True, na=False))) & (main_data['has_water'].isnull())
main_data.loc[water_true, 'has_water'] = True

#---------------true has_electricity---------------
electricity_true = main_data['description'].str.contains(r'\bبرق\b', regex=True, na=False) & (main_data['has_electricity'].isnull())
main_data.loc[electricity_true, 'has_electricity'] = True

#---------------true has_gas---------------
gas_true = main_data['description'].str.contains(r'\bگاز\b', regex=True, na=False) & (main_data['has_gas'].isnull())
main_data.loc[gas_true, 'has_gas'] = True


ایجاد یک تابع برای پر کردن بقیه مقادیر خالی ستون‌های بالا

In [22]:
#df --> data frame
#columns_name --> list
#value --> Any
def fill_null(df, columns_name, value):
    for i in columns_name:
        df[i].fillna(value, inplace = True)

پر کردن بقیه مقادیر ستون‌های بالا

In [23]:

columns_False = ['has_sauna','has_jacuzzi','has_pool','has_barbecue',
                 'has_security_guard','has_business_deed',
                 'has_balcony','has_elevator','is_rebuilt',
                 'has_parking','has_warehouse','has_gas',
                 'has_electricity', 'has_water']
fill_null(main_data, columns_False, False)

C:\Users\ghase\AppData\Local\Temp\ipykernel_17576\548776481.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[i].fillna(value, inplace = True)
C:\Users\ghase\AppData\Local\Temp\ipykernel_17576\548776481.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[i].fillna(value, inplace = True)


In [24]:
main_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 62 columns):
 #   Column                      Non-Null Count    Dtype         
---  ------                      --------------    -----         
 0   Unnamed: 0                  1000000 non-null  int64         
 1   cat2_slug                   1000000 non-null  object        
 2   cat3_slug                   1000000 non-null  object        
 3   city_slug                   1000000 non-null  object        
 4   neighborhood_slug           1000000 non-null  object        
 5   created_at_month            1000000 non-null  datetime64[ns]
 6   user_type                   288882 non-null   object        
 7   description                 1000000 non-null  object        
 8   title                       999946 non-null   object        
 9   rent_mode                   352994 non-null   object        
 10  rent_value                  351322 non-null   float64       
 11  rent_to_single           

# پر کردن صحیح مقادیر تا حد امکان

has_cooling_system

In [25]:
water_cooler_true = (main_data['description'].str.contains(r'\bکولر آبی\b', regex=True, na=False) | (main_data['description'].str.contains(r'\bکولر ابی\b', regex=True, na=False))) & (main_data['has_cooling_system'].isnull())
main_data.loc[water_cooler_true, 'has_cooling_system'] = 'water_cooler'

air_conditioner_true = main_data['description'].str.contains(r'\bتهویه هوا\b', regex=True, na=False) & (main_data['has_cooling_system'].isnull())
main_data.loc[air_conditioner_true, 'has_cooling_system'] = 'air_conditioner'

duct_split_true = main_data['description'].str.contains(r'\bداکت اسپلیت\b', regex=True, na=False) & (main_data['has_cooling_system'].isnull())
main_data.loc[duct_split_true, 'has_cooling_system'] = 'duct_split'

split_true = (main_data['description'].str.contains(r'\bکولر گازی\b', regex=True, na=False) | (main_data['description'].str.contains(r'\bاسپلیت\b', regex=True, na=False))) & (main_data['has_cooling_system'].isnull())
main_data.loc[split_true, 'has_cooling_system'] = 'split'

fan_coil_true = main_data['description'].str.contains(r'\bفن کوئل\b', regex=True, na=False) & (main_data['has_cooling_system'].isnull())
main_data.loc[fan_coil_true, 'has_cooling_system'] = 'fan_coil'

has_heating_system

In [26]:
floor_heating_true = main_data['description'].str.contains(r'\bگرمایش از کف\b', regex=True, na=False) & (main_data['has_heating_system'].isnull())
main_data.loc[floor_heating_true, 'has_heating_system'] = 'floor_heating'

heater_true = main_data['description'].str.contains(r'\bهیتر\b', regex=True, na=False) & (main_data['has_heating_system'].isnull())
main_data.loc[heater_true, 'has_heating_system'] = 'heater'

duct_split_true = main_data['description'].str.contains(r'\bداکت اسپلیت\b', regex=True, na=False) & (main_data['has_heating_system'].isnull())
main_data.loc[duct_split_true, 'has_heating_system'] = 'duct_split'

split_true = ((main_data['description'].str.contains(r'\bاسپلیت\b', regex=True, na=False))) & (main_data['has_heating_system'].isnull())
main_data.loc[split_true, 'has_heating_system'] = 'split'

shoofaj_true = main_data['description'].str.contains(r'\bشوفاژ\b', regex=True, na=False) & (main_data['has_heating_system'].isnull())
main_data.loc[shoofaj_true, 'has_heating_system'] = 'shoofaj'

fan_coil_true = main_data['description'].str.contains(r'\bفن کوئل\b', regex=True, na=False) & (main_data['has_heating_system'].isnull())
main_data.loc[fan_coil_true, 'has_heating_system'] = 'fan_coil'

has_warm_water_provider

In [27]:
package_true = main_data['description'].str.contains(r'\bپکیج\b', regex=True, na=False) & (main_data['has_warm_water_provider'].isnull())
main_data.loc[package_true, 'has_warm_water_provider'] = 'package'

water_heater_1_true = (main_data['description'].str.contains(r'\bآب گرمکن\b', regex=True, na=False) | (main_data['description'].str.contains(r'\bاب گرمکن\b', regex=True, na=False))) & (main_data['has_warm_water_provider'].isnull())
main_data.loc[water_heater_1_true, 'has_warm_water_provider'] = 'water_heater'

water_heater_2_true = (main_data['description'].str.contains(r'\bآبگرمکن\b', regex=True, na=False) | (main_data['description'].str.contains(r'\bابگرمکن\b', regex=True, na=False))) & (main_data['has_warm_water_provider'].isnull())
main_data.loc[water_heater_2_true, 'has_warm_water_provider'] = 'water_heater'

powerhouse_true = (main_data['description'].str.contains(r'\bموتور خونه\b', regex=True, na=False) | (main_data['description'].str.contains(r'\bموتورخونه\b', regex=True, na=False))) & (main_data['has_warm_water_provider'].isnull())
main_data.loc[powerhouse_true, 'has_warm_water_provider'] = 'powerhouse'

بعضی از ستون‌ها مقداری به نام "انتخاب نشده" دارن و ما میتوانیم مقادیر خالی آنها را همین قرار دهیم

In [28]:
unselecteds = ['floor_material','building_direction','has_cooling_system','has_heating_system','has_restroom','has_warm_water_provider','deed_type']
fill_null(main_data, unselecteds, 'unselect')

C:\Users\ghase\AppData\Local\Temp\ipykernel_17576\548776481.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[i].fillna(value, inplace = True)


In [29]:
main_data['has_warm_water_provider'].value_counts()

has_warm_water_provider
unselect        594640
package         228596
water_heater    145238
powerhouse       31526
Name: count, dtype: int64

In [30]:
main_data['has_heating_system'].value_counts()

has_heating_system
unselect         624679
shoofaj          205435
heater           119847
duct_split        22547
floor_heating      9223
split              8997
fan_coil           5222
fireplace          4050
Name: count, dtype: int64

In [31]:
main_data['has_cooling_system'].value_counts()

has_cooling_system
unselect           638180
water_cooler       226331
air_conditioner     52450
split               48026
duct_split          29853
fan_coil             5160
Name: count, dtype: int64

برخی ستون‌ها با اعداد فارسی پر شده‌اند و ما برای تبدیل اونها یک تابع میسازیم

In [32]:
def numbers_persian_to_english(persian_number):
    persian_numbers = '۰۱۲۳۴۵۶۷۸۹'
    english_numbers = '0123456789'

    transform = str.maketrans(persian_numbers, english_numbers)
    
    return str(persian_number).translate(transform)


نویز گیری و تغییر اعداد از انگلیسی به فارسی

In [33]:
main_data['construction_year'] = main_data['construction_year'].str.replace('قبل از ۱۳۷۰', '1369', regex=False)
main_data['construction_year'] = main_data['construction_year'].apply(numbers_persian_to_english)

In [34]:
main_data['construction_year'].value_counts()

construction_year
nan     184172
1403    116260
1390     59139
1402     58424
1400     53674
1395     53029
1398     38207
1397     36326
1396     35487
1401     35328
1385     34065
1399     29594
1393     29094
1392     26130
1394     26110
1388     24268
1380     23480
1369     20637
1389     16755
1391     16316
1387     14136
1386     13468
1383      9894
1384      8494
1375      7247
1382      6965
1371      5531
1381      3590
1378      3025
1379      2415
1377      2117
1372      1914
1373      1827
1376      1593
1374      1289
Name: count, dtype: int64

In [35]:
main_data.columns

Index(['Unnamed: 0', 'cat2_slug', 'cat3_slug', 'city_slug',
       'neighborhood_slug', 'created_at_month', 'user_type', 'description',
       'title', 'rent_mode', 'rent_value', 'rent_to_single', 'rent_type',
       'price_mode', 'price_value', 'credit_mode', 'credit_value',
       'rent_credit_transform', 'transformable_price', 'transformable_credit',
       'transformed_credit', 'transformable_rent', 'transformed_rent',
       'land_size', 'building_size', 'deed_type', 'has_business_deed', 'floor',
       'rooms_count', 'total_floors_count', 'unit_per_floor', 'has_balcony',
       'has_elevator', 'has_warehouse', 'has_parking', 'construction_year',
       'is_rebuilt', 'has_water', 'has_warm_water_provider', 'has_electricity',
       'has_gas', 'has_heating_system', 'has_cooling_system', 'has_restroom',
       'has_security_guard', 'has_barbecue', 'building_direction', 'has_pool',
       'has_jacuzzi', 'has_sauna', 'floor_material', 'property_type',
       'regular_person_capacity',

In [36]:
drop_list =['Unnamed: 0', 'cat2_slug','description', 'rent_price_on_regular_days',
            'rent_to_single', 'property_type','regular_person_capacity','title',
            'extra_person_capacity','cost_per_extra_person', 'user_type',
            'rent_price_on_special_days', 'rent_price_at_weekends','location_radius','unit_per_floor','total_floors_count']

main_data.drop(drop_list, axis= 1, inplace= True)

In [37]:
main_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 46 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   cat3_slug                1000000 non-null  object        
 1   city_slug                1000000 non-null  object        
 2   neighborhood_slug        1000000 non-null  object        
 3   created_at_month         1000000 non-null  datetime64[ns]
 4   rent_mode                352994 non-null   object        
 5   rent_value               351322 non-null   float64       
 6   rent_type                103961 non-null   object        
 7   price_mode               573606 non-null   object        
 8   price_value              568346 non-null   float64       
 9   credit_mode              352994 non-null   object        
 10  credit_value             352095 non-null   float64       
 11  rent_credit_transform    352985 non-null   object        
 12  t

# فهمیدیم که ستون بالکن نویز داره

### پس یه تابع مینویسیم برای نویز گیری و تبدیل کردن مقادیر به بولین وهمچنین این تابع را طوری مینویسم که بتوانیم بقیه موارد را هم به بولین تبدیل کنیم

In [38]:
def to_bool(val):
    if isinstance(val, bool):
        return val
    if isinstance(val, str):
        val = val.strip().lower()
        if val in ['true','water_cooler','air_conditioner','split','duct_split','fan_coil',
                   'shoofaj','heater','duct_split','floor_heating','split','fan_coil','fireplace',
                   'package','water_heater','powerhouse']:
            return True
        elif val in ['false','unselect']:
            return False

## ایجاد ستونی هایی برای سبک تر شدن دیتاست

### بولین کردن ستون ها

In [39]:
main_data['has_balcony'] = main_data['has_balcony'].apply(to_bool)
main_data['has_cooling_system'] = main_data['has_cooling_system'].apply(to_bool)
main_data['has_heating_system'] = main_data['has_heating_system'].apply(to_bool)
main_data['has_warm_water_provider'] = main_data['has_warm_water_provider'].apply(to_bool)

In [40]:
main_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 46 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   cat3_slug                1000000 non-null  object        
 1   city_slug                1000000 non-null  object        
 2   neighborhood_slug        1000000 non-null  object        
 3   created_at_month         1000000 non-null  datetime64[ns]
 4   rent_mode                352994 non-null   object        
 5   rent_value               351322 non-null   float64       
 6   rent_type                103961 non-null   object        
 7   price_mode               573606 non-null   object        
 8   price_value              568346 non-null   float64       
 9   credit_mode              352994 non-null   object        
 10  credit_value             352095 non-null   float64       
 11  rent_credit_transform    352985 non-null   object        
 12  t

### ستون هایی با تایپ بولین را در دسته هایی قرار میدهیم و تعداد مقادیر صحیح بودن هر ستون مربوط به دسته را در ستون جدید ایجاد شده قرار میدهیم

In [41]:
main_data['luxury'] = main_data[['has_sauna', 'has_jacuzzi', 'has_pool', 'has_security_guard', 'has_barbecue']].sum(axis=1)
main_data['necessaries'] = main_data[['has_gas', 'has_water', 'has_electricity']].sum(axis=1)
main_data['basic'] = main_data[['has_balcony', 'has_elevator', 'has_warehouse', 'has_parking']].sum(axis=1)
main_data['extra_features'] = main_data[['has_warm_water_provider', 'has_heating_system', 'has_cooling_system']].sum(axis=1)

### ستون های اضافه را حذف میکنیم

#### نکته: درسته ستون 'ضروری ها' رو درست کردیم ولی از اونجایی که با دانش عمومی ما که تقریبا همه خون ها این امکانات رو دارن مغایرت داره پس حذفش میکنیم

In [42]:
drop_list2 = ['has_sauna', 'has_jacuzzi', 'has_pool', 'has_security_guard', 'has_barbecue',
              'has_gas', 'has_water', 'has_electricity',
              'has_balcony', 'has_elevator', 'has_warehouse', 'has_parking',
              'has_warm_water_provider', 'has_heating_system', 'has_cooling_system',
              'has_restroom', 'building_direction', 'floor_material',
              'necessaries']

main_data.drop(drop_list2, axis= 1, inplace= True)

In [43]:
main_data.describe()

,created_at_month,rent_value,price_value,credit_value,transformable_credit,transformed_credit,transformable_rent,transformed_rent,land_size,building_size,location_latitude,location_longitude,big_city,luxury,basic,extra_features
count,1000000,3.513220e+05,5.683460e+05,3.520950e+05,3.520850e+05,7.240900e+04,3.512480e+05,7.240900e+04,1.000000e+06,1.000000e+06,655608.000000,655608.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,2024-08-14 02:08:19.622400256,4.102299e+10,1.736537e+10,4.872084e+10,4.872222e+10,8.557025e+09,4.103164e+10,1.619934e+07,7.012465e+03,4.353565e+03,34.982108,51.629743,0.464799,0.125686,1.997441,1.142501
min,2020-02-01 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-1.000000e+00,23.626478,40.162369,0.000000,0.000000,0.000000,0.000000
25%,2024-07-01 00:00:00,1.111110e+05,1.400000e+09,1.000000e+08,1.000000e+08,2.000000e+08,1.111110e+05,1.000000e+06,1.166667e+02,7.300000e+01,34.553551,50.677175,0.000000,0.000000,0.000000,0.000000
50%,2024-08-01 00:00:00,5.000000e+06,2.840000e+09,2.500000e+08,2.500000e+08,4.000000e+08,5.000000e+06,6.000000e+06,1.666667e+02,1.000000e+02,35.723312,51.345791,0.000000,0.000000,2.000000,0.000000
75%,2024-10-01 00:00:00,1.200000e+07,5.900000e+09,5.000000e+08,5.000000e+08,8.500000e+08,1.200000e+07,1.500000e+07,2.666667e+02,1.620000e+02,36.307013,51.805291,1.000000,0.000000,3.000000,3.000000
max,2025-03-01 00:00:00,1.000000e+15,1.000000e+14,1.000000e+15,1.000000e+15,5.555556e+14,1.000000e+15,3.000000e+09,1.666667e+07,1.000000e+07,40.358055,74.511620,1.000000,5.000000,4.000000,3.000000
std,NaN,3.807534e+12,5.878739e+11,4.341346e+12,4.341407e+12,2.064576e+12,3.807935e+12,5.217890e+07,2.157586e+05,1.353664e+05,2.379169,3.160920,0.498760,0.495398,1.506989,1.373927


In [44]:
main_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 31 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   cat3_slug              1000000 non-null  object        
 1   city_slug              1000000 non-null  object        
 2   neighborhood_slug      1000000 non-null  object        
 3   created_at_month       1000000 non-null  datetime64[ns]
 4   rent_mode              352994 non-null   object        
 5   rent_value             351322 non-null   float64       
 6   rent_type              103961 non-null   object        
 7   price_mode             573606 non-null   object        
 8   price_value            568346 non-null   float64       
 9   credit_mode            352994 non-null   object        
 10  credit_value           352095 non-null   float64       
 11  rent_credit_transform  352985 non-null   object        
 12  transformable_price    352894

# ایجاد ستون طول عرض جغرافیایی به جای اسم شهر

In [45]:
copy_main = main_data.copy()

## جدا کردن قسمت صحیح عدد

In [46]:

copy_main['lat'] = copy_main['location_latitude'].floordiv(1).astype('Int64')
copy_main['long'] = copy_main['location_longitude'].floordiv(1).astype('Int64')


In [47]:
data_with_long = copy_main[copy_main['long'].notnull()].copy()

In [48]:
data_with_long_unique = data_with_long.drop_duplicates(subset='city_slug', keep='first').copy()

### حذف متغیرهای بی‌استفاده برای کمتر کردن فضای مصرفی

In [49]:
del(copy_main)
del(data_with_long)

In [50]:
data_with_long_unique

,cat3_slug,city_slug,neighborhood_slug,created_at_month,rent_mode,rent_value,rent_type,price_mode,price_value,credit_mode,...,construction_year,is_rebuilt,location_latitude,location_longitude,big_city,luxury,basic,extra_features,lat,long
0,villa,karaj,mehrshahr,2024-08-01,NaN,NaN,NaN,NaN,NaN,NaN,...,nan,False,35.811684,50.936600,1,1,0,0,35,50
2,apartment-rent,tehran,tohid,2024-10-01,مقطوع,26000000.0,NaN,NaN,NaN,مقطوع,...,1401,False,35.703865,51.373459,1,0,3,0,35,51
8,apartment-sell,mahdasht-city,mahdasht-city,2024-06-01,NaN,NaN,NaN,مقطوع,6.500000e+08,NaN,...,1396,True,35.712364,50.794781,0,0,4,3,35,50
10,apartment-sell,pardis-city,pardis-city,2024-11-01,NaN,NaN,NaN,مقطوع,2.600000e+09,NaN,...,1403,True,35.778664,51.757549,0,0,4,3,35,51
12,apartment-sell,mashhad,farhang,2024-06-01,NaN,NaN,NaN,مقطوع,1.299000e+10,NaN,...,1403,False,36.321831,59.524502,1,0,4,3,36,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244237,house-villa-sell,yanghagh,yanghagh,2024-08-01,NaN,NaN,NaN,مقطوع,4.444444e+06,NaN,...,1393,False,37.299164,55.374279,0,0,3,3,37,55
311752,plot-old,sijval,sijval,2024-08-01,NaN,NaN,NaN,مقطوع,3.000000e+08,NaN,...,nan,False,36.903152,54.194168,0,0,0,0,36,54
449425,villa,khoour,khoour,2024-10-01,NaN,NaN,NaN,NaN,NaN,NaN,...,nan,False,36.794621,53.066551,0,0,0,0,36,53
468103,plot-old,maraveh-tapeh,maraveh-tapeh,2024-10-01,NaN,NaN,NaN,مقطوع,9.000000e+11,NaN,...,nan,False,37.811443,56.025665,0,0,0,0,37,56


### ادغام کردم دو دیتا فریم برای داشتم دو ستون جدید

In [51]:
df_big = main_data.merge(
    data_with_long_unique[['city_slug', 'lat', 'long']], 
    on='city_slug',                          
    how='left'                               
)

In [52]:
main_data

,cat3_slug,city_slug,neighborhood_slug,created_at_month,rent_mode,rent_value,rent_type,price_mode,price_value,credit_mode,...,floor,rooms_count,construction_year,is_rebuilt,location_latitude,location_longitude,big_city,luxury,basic,extra_features
0,villa,karaj,mehrshahr,2024-08-01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,سه,nan,False,35.811684,50.936600,1,1,0,0
1,apartment-sell,tehran,gholhak,2024-05-01,NaN,NaN,NaN,مقطوع,8.500000e+09,NaN,...,3,یک,1384,False,NaN,NaN,1,0,3,0
2,apartment-rent,tehran,tohid,2024-10-01,مقطوع,26000000.0,NaN,NaN,NaN,مقطوع,...,3,سه,1401,False,35.703865,51.373459,1,0,3,0
3,office-rent,tehran,elahiyeh,2024-06-01,مقطوع,95000000.0,NaN,NaN,NaN,مقطوع,...,4,یک,1400,False,NaN,NaN,1,1,2,0
4,apartment-sell,mashhad,emamreza,2024-05-01,NaN,NaN,NaN,مقطوع,5.750000e+09,NaN,...,4,دو,1403,False,NaN,NaN,1,0,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,apartment-sell,kermanshah,unknown,2024-07-01,NaN,NaN,NaN,مقطوع,7.470000e+09,NaN,...,4.0,چهار,1403,False,34.350235,47.083241,1,0,3,0
999996,apartment-rent,tehran,darya,2024-07-01,مقطوع,45000000.0,rent_credit,NaN,NaN,مقطوع,...,1.0,دو,1403,False,35.770454,51.369099,1,0,3,0
999997,house-villa-sell,yazd,yazd,2024-11-01,NaN,NaN,NaN,مقطوع,3.200000e+09,NaN,...,NaN,چهار,1369,False,NaN,NaN,0,0,3,0
999998,suite-apartment,bandar-anzali,bandar-anzali,2024-09-01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,دو,nan,False,37.483501,49.438721,0,0,0,0


In [53]:
df_big

,cat3_slug,city_slug,neighborhood_slug,created_at_month,rent_mode,rent_value,rent_type,price_mode,price_value,credit_mode,...,construction_year,is_rebuilt,location_latitude,location_longitude,big_city,luxury,basic,extra_features,lat,long
0,villa,karaj,mehrshahr,2024-08-01,NaN,NaN,NaN,NaN,NaN,NaN,...,nan,False,35.811684,50.936600,1,1,0,0,35,50
1,apartment-sell,tehran,gholhak,2024-05-01,NaN,NaN,NaN,مقطوع,8.500000e+09,NaN,...,1384,False,NaN,NaN,1,0,3,0,35,51
2,apartment-rent,tehran,tohid,2024-10-01,مقطوع,26000000.0,NaN,NaN,NaN,مقطوع,...,1401,False,35.703865,51.373459,1,0,3,0,35,51
3,office-rent,tehran,elahiyeh,2024-06-01,مقطوع,95000000.0,NaN,NaN,NaN,مقطوع,...,1400,False,NaN,NaN,1,1,2,0,35,51
4,apartment-sell,mashhad,emamreza,2024-05-01,NaN,NaN,NaN,مقطوع,5.750000e+09,NaN,...,1403,False,NaN,NaN,1,0,4,3,36,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,apartment-sell,kermanshah,unknown,2024-07-01,NaN,NaN,NaN,مقطوع,7.470000e+09,NaN,...,1403,False,34.350235,47.083241,1,0,3,0,34,47
999996,apartment-rent,tehran,darya,2024-07-01,مقطوع,45000000.0,rent_credit,NaN,NaN,مقطوع,...,1403,False,35.770454,51.369099,1,0,3,0,35,51
999997,house-villa-sell,yazd,yazd,2024-11-01,NaN,NaN,NaN,مقطوع,3.200000e+09,NaN,...,1369,False,NaN,NaN,0,0,3,0,31,54
999998,suite-apartment,bandar-anzali,bandar-anzali,2024-09-01,NaN,NaN,NaN,NaN,NaN,NaN,...,nan,False,37.483501,49.438721,0,0,0,0,37,49


In [54]:
df_big.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 33 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   cat3_slug              1000000 non-null  object        
 1   city_slug              1000000 non-null  object        
 2   neighborhood_slug      1000000 non-null  object        
 3   created_at_month       1000000 non-null  datetime64[ns]
 4   rent_mode              352994 non-null   object        
 5   rent_value             351322 non-null   float64       
 6   rent_type              103961 non-null   object        
 7   price_mode             573606 non-null   object        
 8   price_value            568346 non-null   float64       
 9   credit_mode            352994 non-null   object        
 10  credit_value           352095 non-null   float64       
 11  rent_credit_transform  352985 non-null   object        
 12  transformable_price    352894

# فیلتر کردن سطرها

## سطر هایی با مقادیر خالی در ستون مختصات را حذف میکنیم

In [55]:
df_big = df_big.dropna(subset=['location_latitude', 'location_longitude'])

### به صورت دستی بیشترین و کمترین مقدار در ستون های طول و عرض مختصاتی را متناسب با کشورمان تنظیم میکنیم

In [56]:
df_big = df_big[(df_big['location_latitude'] >= 25.8) & (df_big['location_latitude'] <= 47)]


In [57]:
df_big = df_big[(df_big['location_longitude'] >= 44.2) & (df_big['location_longitude'] <= 61)]

# پیدا کردن داده های پرت در ستون های اندازه زمین و اندازه ساختمان

## برای پیدا کردن داده های پرت در ستون های اندازه زمین و اندازه ساختمان اول به صورت دستی یک حد پایین تعریف میکنیم

In [58]:
df_big = df_big[(df_big['land_size'] >= 25)]

In [59]:
df_big = df_big[(df_big['building_size'] >= 10)]

In [60]:
df_big.info()

<class 'pandas.core.frame.DataFrame'>
Index: 634884 entries, 0 to 999999
Data columns (total 33 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   cat3_slug              634884 non-null  object        
 1   city_slug              634884 non-null  object        
 2   neighborhood_slug      634884 non-null  object        
 3   created_at_month       634884 non-null  datetime64[ns]
 4   rent_mode              230206 non-null  object        
 5   rent_value             229966 non-null  float64       
 6   rent_type              64331 non-null   object        
 7   price_mode             369477 non-null  object        
 8   price_value            367348 non-null  float64       
 9   credit_mode            230206 non-null  object        
 10  credit_value           230104 non-null  float64       
 11  rent_credit_transform  230199 non-null  object        
 12  transformable_price    230188 non-null  object   

## داده های پرت را به این روش پیدا و حذف میکنیم

In [61]:
Q1 = df_big['land_size'].quantile(0.25)
Q3 = df_big['land_size'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_big = df_big[(df_big['land_size'] >= lower_bound) & (df_big['land_size'] <= upper_bound)]

In [62]:
Q1 = df_big['building_size'].quantile(0.25)
Q3 = df_big['building_size'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df = df_big[(df_big['building_size'] >= lower_bound) & (df_big['building_size'] <= upper_bound)]

In [63]:
df

,cat3_slug,city_slug,neighborhood_slug,created_at_month,rent_mode,rent_value,rent_type,price_mode,price_value,credit_mode,...,construction_year,is_rebuilt,location_latitude,location_longitude,big_city,luxury,basic,extra_features,lat,long
2,apartment-rent,tehran,tohid,2024-10-01,مقطوع,26000000.0,NaN,NaN,NaN,مقطوع,...,1401,False,35.703865,51.373459,1,0,3,0,35,51
7,apartment-sell,tehran,dardasht,2024-09-01,NaN,NaN,NaN,مقطوع,8.700000e+09,NaN,...,1393,False,35.729832,51.505466,1,0,3,0,35,51
8,apartment-sell,mahdasht-city,mahdasht-city,2024-06-01,NaN,NaN,NaN,مقطوع,6.500000e+08,NaN,...,1396,True,35.712364,50.794781,0,0,4,3,35,50
10,apartment-sell,pardis-city,pardis-city,2024-11-01,NaN,NaN,NaN,مقطوع,2.600000e+09,NaN,...,1403,True,35.778664,51.757549,0,0,4,3,35,51
11,apartment-rent,tehran,gisha,2024-08-01,مجانی,0.0,rent_credit,NaN,NaN,مقطوع,...,1385,True,35.733952,51.380608,1,0,3,0,35,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999993,apartment-rent,mashhad,elahiyehblvd,2024-06-01,مقطوع,2000000.0,rent_credit,NaN,NaN,مقطوع,...,1386,True,36.374050,59.485794,1,0,2,1,36,59
999995,apartment-sell,kermanshah,unknown,2024-07-01,NaN,NaN,NaN,مقطوع,7.470000e+09,NaN,...,1403,False,34.350235,47.083241,1,0,3,0,34,47
999996,apartment-rent,tehran,darya,2024-07-01,مقطوع,45000000.0,rent_credit,NaN,NaN,مقطوع,...,1403,False,35.770454,51.369099,1,0,3,0,35,51
999998,suite-apartment,bandar-anzali,bandar-anzali,2024-09-01,NaN,NaN,NaN,NaN,NaN,NaN,...,nan,False,37.483501,49.438721,0,0,0,0,37,49


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 531025 entries, 2 to 999999
Data columns (total 33 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   cat3_slug              531025 non-null  object        
 1   city_slug              531025 non-null  object        
 2   neighborhood_slug      531025 non-null  object        
 3   created_at_month       531025 non-null  datetime64[ns]
 4   rent_mode              215096 non-null  object        
 5   rent_value             214898 non-null  float64       
 6   rent_type              59896 non-null   object        
 7   price_mode             297081 non-null  object        
 8   price_value            296330 non-null  float64       
 9   credit_mode            215096 non-null  object        
 10  credit_value           215027 non-null  float64       
 11  rent_credit_transform  215091 non-null  object        
 12  transformable_price    215084 non-null  object   

# تغییر دادن مختصات به حالت استاندارد

In [65]:

import utm 

def latlon_to_utm(latitude, longitude):
    try:
        easting, northing, zone_number, zone_letter = utm.from_latlon(latitude, longitude)
        return easting,northing,zone_number,zone_letter
    
    except Exception as e:
        return  None, None, None, None


df[['easting', 'northing', 'zone_number', 'zone_letter']] = df.apply(
    lambda row: pd.Series(latlon_to_utm(row['location_latitude'], row['location_longitude'])), axis=1)


C:\Users\ghase\AppData\Local\Temp\ipykernel_17576\2862628650.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['easting', 'northing', 'zone_number', 'zone_letter']] = df.apply(
C:\Users\ghase\AppData\Local\Temp\ipykernel_17576\2862628650.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['easting', 'northing', 'zone_number', 'zone_letter']] = df.apply(
C:\Users\ghase\AppData\Local\Temp\ipykernel_17576\2862628650.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [66]:
df.describe()

,created_at_month,rent_value,price_value,credit_value,transformable_credit,transformed_credit,transformable_rent,transformed_rent,land_size,building_size,...,location_longitude,big_city,luxury,basic,extra_features,lat,long,easting,northing,zone_number
count,531025,2.148980e+05,2.963300e+05,2.150270e+05,2.150240e+05,5.144500e+04,2.148900e+05,5.144500e+04,531025.000000,531025.000000,...,531025.000000,531025.000000,531025.000000,531025.000000,531025.000000,531025.0,531025.0,531025.000000,5.310250e+05,531025.000000
mean,2024-08-19 07:14:40.833105664,2.752544e+10,1.414677e+10,3.214277e+10,3.214322e+10,1.144808e+10,2.752646e+10,1.113828e+07,165.018504,101.205926,...,51.620625,0.501462,0.089811,2.298353,1.445602,34.456118,51.129521,519536.805788,3.877840e+06,39.067260
min,2022-04-01 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,25.000000,10.000000,...,44.253368,0.000000,0.000000,0.000000,0.000000,25.0,44.0,200900.869520,2.857632e+06,38.000000
25%,2024-07-01 00:00:00,1.000000e+05,1.550000e+09,1.000000e+08,1.000000e+08,2.000000e+08,1.000000e+05,1.000000e+06,110.000000,70.000000,...,50.830765,0.000000,0.000000,1.000000,0.000000,34.0,50.0,491250.026522,3.850471e+06,39.000000
50%,2024-09-01 00:00:00,5.000000e+06,2.900000e+09,2.500000e+08,2.500000e+08,4.000000e+08,5.000000e+06,6.000000e+06,150.000000,92.000000,...,51.348526,1.000000,0.000000,3.000000,1.000000,35.0,51.0,531994.292831,3.953193e+06,39.000000
75%,2024-10-01 00:00:00,1.150000e+07,5.500000e+09,5.000000e+08,5.000000e+08,8.000000e+08,1.150000e+07,1.350000e+07,206.666667,126.000000,...,51.749454,1.000000,0.000000,3.000000,3.000000,36.0,51.0,568199.144387,4.013366e+06,39.000000
max,2025-03-01 00:00:00,1.000000e+15,1.000000e+14,1.000000e+15,1.000000e+15,5.555556e+14,1.000000e+15,5.600000e+08,489.000000,232.000000,...,60.997467,1.000000,5.000000,4.000000,3.000000,39.0,62.0,799465.654518,4.468484e+06,41.000000
std,NaN,2.662368e+12,4.803376e+11,3.477507e+12,3.477532e+12,2.449376e+12,2.662418e+12,1.897365e+07,77.150158,45.349115,...,3.077937,0.499998,0.405120,1.424253,1.409205,2.315878,3.082297,118365.990279,2.514034e+05,0.482455


In [67]:
df

,cat3_slug,city_slug,neighborhood_slug,created_at_month,rent_mode,rent_value,rent_type,price_mode,price_value,credit_mode,...,big_city,luxury,basic,extra_features,lat,long,easting,northing,zone_number,zone_letter
2,apartment-rent,tehran,tohid,2024-10-01,مقطوع,26000000.0,NaN,NaN,NaN,مقطوع,...,1,0,3,0,35,51,533784.424602,3.951168e+06,39,S
7,apartment-sell,tehran,dardasht,2024-09-01,NaN,NaN,NaN,مقطوع,8.700000e+09,NaN,...,1,0,3,0,35,51,545711.558185,3.954101e+06,39,S
8,apartment-sell,mahdasht-city,mahdasht-city,2024-06-01,NaN,NaN,NaN,مقطوع,6.500000e+08,NaN,...,0,0,4,3,35,50,481437.130969,3.952066e+06,39,S
10,apartment-sell,pardis-city,pardis-city,2024-11-01,NaN,NaN,NaN,مقطوع,2.600000e+09,NaN,...,0,0,4,3,35,51,568467.028494,3.959664e+06,39,S
11,apartment-rent,tehran,gisha,2024-08-01,مجانی,0.0,rent_credit,NaN,NaN,مقطوع,...,1,0,3,0,35,51,534418.187237,3.954507e+06,39,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999993,apartment-rent,mashhad,elahiyehblvd,2024-06-01,مقطوع,2000000.0,rent_credit,NaN,NaN,مقطوع,...,1,0,2,1,36,59,722996.875179,4.028307e+06,40,S
999995,apartment-sell,kermanshah,unknown,2024-07-01,NaN,NaN,NaN,مقطوع,7.470000e+09,NaN,...,1,0,3,0,34,47,691605.054490,3.802956e+06,38,S
999996,apartment-rent,tehran,darya,2024-07-01,مقطوع,45000000.0,rent_credit,NaN,NaN,مقطوع,...,1,0,3,0,35,51,533362.195046,3.958552e+06,39,S
999998,suite-apartment,bandar-anzali,bandar-anzali,2024-09-01,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,37,49,361963.574835,4.149656e+06,39,S


In [68]:
df.columns

Index(['cat3_slug', 'city_slug', 'neighborhood_slug', 'created_at_month',
       'rent_mode', 'rent_value', 'rent_type', 'price_mode', 'price_value',
       'credit_mode', 'credit_value', 'rent_credit_transform',
       'transformable_price', 'transformable_credit', 'transformed_credit',
       'transformable_rent', 'transformed_rent', 'land_size', 'building_size',
       'deed_type', 'has_business_deed', 'floor', 'rooms_count',
       'construction_year', 'is_rebuilt', 'location_latitude',
       'location_longitude', 'big_city', 'luxury', 'basic', 'extra_features',
       'lat', 'long', 'easting', 'northing', 'zone_number', 'zone_letter'],
      dtype='object')

In [69]:
df['construction_year'].value_counts()

construction_year
1403    65965
nan     47337
1390    35784
1402    32048
1395    31662
1400    30236
1398    22363
1397    21578
1396    21467
1385    21444
1401    20483
1393    18479
1399    16809
1392    16439
1394    16383
1388    15501
1380    13764
1389    10585
1369    10340
1391    10233
1387     9015
1386     8577
1383     6668
1384     5507
1382     4612
1375     4169
1371     2977
1381     2340
1378     1910
1379     1481
1377     1203
1373     1044
1372     1004
1376      921
1374      697
Name: count, dtype: int64

# تبدیل قیمت اجاره و رهن به قیمت ارزش ساختمان و ساخت ستون هزینه واقعی

## پر کردن مقادیر خالی دو ستون مقدار رهن و اجاره و تبدیل رهن به اجاره

In [70]:

df['credit_value'] = df['credit_value'].fillna(0)
df['rent_value'] = df['rent_value'].fillna(0)


conversion_rate = 0.04


df['total_credit_estimate'] = df['credit_value'] + (df['rent_value'] / conversion_rate)


C:\Users\ghase\AppData\Local\Temp\ipykernel_17576\932993926.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['credit_value'] = df['credit_value'].fillna(0)
C:\Users\ghase\AppData\Local\Temp\ipykernel_17576\932993926.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rent_value'] = df['rent_value'].fillna(0)
C:\Users\ghase\AppData\Local\Temp\ipykernel_17576\932993926.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

## تبدیل قیمت رهن تمام به قیمت ارزش ملک

In [73]:
df['total_credit_estimate'] = df['total_credit_estimate'] * 10

C:\Users\ghase\AppData\Local\Temp\ipykernel_17576\2633954465.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_credit_estimate'] = df['total_credit_estimate'] * 10


## پر کردن مقادیر خالی ارزش ملک و تغییر نام آن

In [75]:
df['price_value'] = df['price_value'].fillna(df['total_credit_estimate'])

C:\Users\ghase\AppData\Local\Temp\ipykernel_17576\2239571445.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_value'] = df['price_value'].fillna(df['total_credit_estimate'])


In [77]:
df.rename(columns={'price_value': 'reall_cost'}, inplace=True)


C:\Users\ghase\AppData\Local\Temp\ipykernel_17576\3021747118.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'price_value': 'reall_cost'}, inplace=True)


## حذف ستون هایی که مربوط به موارد مالی هستند ولی به کار نمی‌آیند

In [78]:
df.columns

Index(['cat3_slug', 'city_slug', 'neighborhood_slug', 'created_at_month',
       'rent_mode', 'rent_value', 'rent_type', 'price_mode', 'reall_cost',
       'credit_mode', 'credit_value', 'rent_credit_transform',
       'transformable_price', 'transformable_credit', 'transformed_credit',
       'transformable_rent', 'transformed_rent', 'land_size', 'building_size',
       'deed_type', 'has_business_deed', 'floor', 'rooms_count',
       'construction_year', 'is_rebuilt', 'location_latitude',
       'location_longitude', 'big_city', 'luxury', 'basic', 'extra_features',
       'lat', 'long', 'easting', 'northing', 'zone_number', 'zone_letter',
       'total_credit_estimate'],
      dtype='object')

In [79]:
df = df.drop(['rent_mode', 'rent_value', 'rent_type', 'price_mode',
       'credit_mode', 'credit_value', 'rent_credit_transform',
       'transformable_price', 'transformable_credit', 'transformed_credit',
       'transformable_rent', 'transformed_rent','total_credit_estimate'],axis=1)

## حذف داده های پرت

In [81]:
Q1 = df['reall_cost'].quantile(0.25)
Q3 = df['reall_cost'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df = df[(df['reall_cost'] >= lower_bound) & (df['reall_cost'] <= upper_bound)]

### گذاشتن یک شرط به صورت دستی

In [83]:
df = df[df['reall_cost'] > 0]

## راحت شدن از دست تورم

### به دست آوردن مقدار تورم برای نرمال سازی

In [88]:
cpi_iran = {
    2022: 0.68,
    2023: 0.80,
    2024: 0.90,
    2025: 1.00,
}

### جدا کردن سال برای نرمال سازی بر اساس سال

In [89]:
df['year'] = df['created_at_month'].dt.year

C:\Users\ghase\AppData\Local\Temp\ipykernel_17576\2775647576.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['created_at_month'].dt.year


### مپ کردن مقدار تورم روی ستون سال و ذخیره توی ستون چدید

In [91]:
df['cpi_ratio'] = df['year'].map(cpi_iran)

C:\Users\ghase\AppData\Local\Temp\ipykernel_17576\3928932315.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cpi_ratio'] = df['year'].map(cpi_iran)


### راحت شدن از دست تورم

In [92]:
df['adjusted_price_value'] = df['reall_cost'] / df['cpi_ratio']

C:\Users\ghase\AppData\Local\Temp\ipykernel_17576\2878802596.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['adjusted_price_value'] = df['reall_cost'] / df['cpi_ratio']


#### حذف ستون های اضافی

In [95]:
df = df.drop(['year','cpi_ratio','created_at_month','reall_cost'],axis=1)

In [96]:
df

,cat3_slug,city_slug,neighborhood_slug,land_size,building_size,deed_type,has_business_deed,floor,rooms_count,construction_year,...,luxury,basic,extra_features,lat,long,easting,northing,zone_number,zone_letter,adjusted_price_value
7,apartment-sell,tehran,dardasht,166.666667,100.0,unselect,False,4,دو,1393,...,0,3,0,35,51,545711.558185,3.954101e+06,39,S,9.666667e+09
8,apartment-sell,mahdasht-city,mahdasht-city,130.000000,78.0,single_page,False,4,دو,1396,...,0,4,3,35,50,481437.130969,3.952066e+06,39,S,7.222222e+08
10,apartment-sell,pardis-city,pardis-city,145.000000,87.0,other,False,4,دو,1403,...,0,4,3,35,51,568467.028494,3.959664e+06,39,S,2.888889e+09
11,apartment-rent,tehran,gisha,151.666667,91.0,unselect,False,1,دو,1385,...,0,3,0,35,51,534418.187237,3.954507e+06,39,S,1.333333e+10
12,apartment-sell,mashhad,farhang,281.666667,169.0,single_page,False,2,سه,1403,...,0,4,3,36,59,726621.357914,4.022603e+06,40,S,1.443333e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999990,apartment-sell,urmia,urmia,233.333333,140.0,unselect,False,1.0,دو,1401,...,0,4,0,37,45,504451.826806,4.150933e+06,38,S,4.444444e+09
999991,apartment-sell,tehran,shaharak-e-moslemin,151.666667,91.0,single_page,False,4.0,دو,1388,...,0,2,3,35,51,529325.784884,3.945410e+06,39,S,4.000000e+09
999993,apartment-rent,mashhad,elahiyehblvd,83.333333,50.0,unselect,False,1.0,یک,1386,...,0,2,1,36,59,722996.875179,4.028307e+06,40,S,2.222222e+09
999995,apartment-sell,kermanshah,unknown,300.000000,180.0,unselect,False,4.0,چهار,1403,...,0,3,0,34,47,691605.054490,3.802956e+06,38,S,8.300000e+09


In [97]:
df.to_csv('final_pre_last.csv', index=False)
